# Eat Safe, Love

## Notebook Set Up

In [63]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [64]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [65]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [66]:
# review the collections in our database


In [67]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [68]:
# Find the establishments with a hygiene score of 20
query ={'scores.hygiene': 20}

# Use count_documents to display the number of documents in the result


# Display the first document in the results using pprint
establishments.find_one(query)

In [69]:
# Convert the result to a Pandas DataFrame
lists = list(establishments.find(query))
estab_df = pd.json_normalize(lists)
# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
estab_df

""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [70]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'RatingValue': {'$gte': 4},
         "LocalAuthorityName": {"$regex": 'City of London Corporation'}}

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint
establishments.find_one(query)

{'_id': ObjectId('6709c8a4d81faff12e360bba'),
 'FHRSID': 621707,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'PostCode': 'ME12',
 'Phone': '',
 'RatingValue': 4,
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2021-10-18T00:00:00',
 'LocalAuthorityCode': '508',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 10, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('0.508551'),
  'latitude': Decimal128('51.369321')},
 'RightToReply': '',
 'Distance': 4627.439467780196,
 'NewRatingPending': False,


In [71]:
# Convert the result to a Pandas DataFrame
lis = establishments.find(query)
esta_df = pd.json_normalize(lis)
# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [76]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.369321
longitude = 0.508551

query = {'RatingValue': 5, 
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search} 
}
sort = [('score.hygiene', 1)]
limit = 5
# Print the results
printed_es = list(establishments.find(query).sort(sort).limit(limit))

for es in printed_es:
    print(es)

res = len(list(establishments.find(query).sort(sort).limit(limit)))
res

{'_id': ObjectId('6709c8a4d81faff12e3609c9'), 'FHRSID': 1423359, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': '21/00411/CP', 'BusinessName': 'Que & Brew Smokehouse', 'BusinessType': 'Takeaway/sandwich shop', 'BusinessTypeID': 7844, 'AddressLine1': 'The Huntsman', 'AddressLine2': '106 Pattens Lane', 'AddressLine3': 'Rochester', 'AddressLine4': 'Medway', 'PostCode': 'ME1 2RA', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-02-20T00:00:00', 'LocalAuthorityCode': '896', 'LocalAuthorityName': 'Medway', 'LocalAuthorityWebSite': 'http://www.medway.gov.uk/', 'LocalAuthorityEmailAddress': 'foodandsafety@medway.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('0.5160458'), 'latitude': Decimal128('51.3666861')}, 'RightToReply': '', 'Distance': 4627.068948009548, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0

5

In [77]:
# Convert result to Pandas DataFrame
top_5 = pd.json_normalize(printed_es)

top_5

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,geocode.longitude,geocode.latitude,meta.dataSource,meta.extractDate,meta.itemCount,meta.returncode,meta.totalCount,meta.totalPages,meta.pageSize,meta.pageNumber
0,6709c8a4d81faff12e3609c9,1423359,0,21/00411/CP,Que & Brew Smokehouse,Takeaway/sandwich shop,7844,The Huntsman,106 Pattens Lane,Rochester,...,0.5160458,51.3666861,None,0001-01-01T00:00:00,0,None,0,0,0,0
1,6709c8a4d81faff12e3609d2,786158,0,PI/000030142,News Shop,Retailers - other,4613,110 Pattens Lane,Rochester,Kent,...,0.516021,51.366737,None,0001-01-01T00:00:00,0,None,0,0,0,0
2,6709c8a4d81faff12e3609c6,510152,0,PI/000243002,The Huntsman,Pub/bar/nightclub,7843,The Huntsman PH,The Huntsman,106 Pattens Lane,...,0.5160458,51.3666861,None,0001-01-01T00:00:00,0,None,0,0,0,0
3,6709c8a4d81faff12e360979,783921,0,PI/000030119,Co-Op,Retailers - other,4613,27-31 Pattens Lane,Chatham,Kent,...,0.517494,51.366103,None,0001-01-01T00:00:00,0,None,0,0,0,0
4,6709c8a4d81faff12e3609c8,802088,0,PI/000251174,Sealand Fish And Chips,Takeaway/sandwich shop,7844,108 Pattens Lane,Rochester,Kent,...,0.516021,51.366737,None,0001-01-01T00:00:00,0,None,0,0,0,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [80]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
matching = {'$match': {'scores.Hygiene': 0}}
grouping ={'$group': {'_id':'$LocalAuthorityName', 
                      'count': {'$sum':1}}}
sorting = {'$sort': {'count': -1}}
pipe = [matching,grouping,sorting]

res = list(establishments.aggregate(pipe))
# Print the number of documents in the result
print(f"docs: {len(res)}")
# Print the first 10 results
pprint(res)


docs: 55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542},
 {'_id': 'Colchester', 'count': 498},
 {'_id': 'Tunbridge Wells', 'count': 491},
 {'_id': 'Folkestone and Hythe', 'count': 480},
 {'_id': 'Eastbourne', 'count': 478},
 {'_id': 'Hastings', 'count': 464},
 {'_id': 'Bromley', 'count': 460},
 {'_id': 'Ashford', 'count': 427},
 {'_id': 'Havering', 'count': 397},
 {'_id': 'Dartford', 'count': 383},
 {'_id': 'Braintree', 'count': 382},
 {'_id': 'Basildon', 'count': 362},
 {'_id': 'Gravesham', 'count': 339},
 {'_id': 'Tonbridge and Malling', 'count': 325},
 {'_id': 'Thurrock', 'count': 313},
 {'_id': 'Barking and Dagenham', 'count': 301},
 {'_id': 'Canterbury City', 'count': 298},

In [81]:
# Convert the result to a Pandas DataFrame
res_df = pd.json_normalize(res)
# Display the number of rows in the DataFrame
print(len(res_df))
# Display the first 10 rows of the DataFrame
res_df.head(10)

55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
